In [61]:
import datetime
import requests
import pandas as pd

In [62]:
codigoINE_renta = {
    "Bizkaia" : 30917,
    "Gipuzkoa" : 31007,
    "Araba" : 30851
}

hours_added = datetime.timedelta(hours = 1)
num_datos = 50000
url_plantilla = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{codigo}?nult={num_datos}'

In [63]:
df_renta = pd.DataFrame(columns=[])

for provincia in codigoINE_renta:
    main_url_INE_Estructura_Datos = "https://servicios.ine.es/wstempus/js/ES/SERIES_TABLA/" + str(codigoINE_renta[provincia]) + "?page=1"
    INE_Estructura_Datos = requests.get(main_url_INE_Estructura_Datos).json()

    df_INE_Estructura_Datos_Renta = pd.DataFrame.from_dict(INE_Estructura_Datos)
    
    new = df_INE_Estructura_Datos_Renta["Nombre"].str.split('.')
    df_INE_Estructura_Datos_Renta["Nombre"] = new.map(lambda x: x[0])
    df_INE_Estructura_Datos_Renta["Tipo Renta"] = new.map(lambda x: x[2])
    
    df_INE_Estructura_Datos_Total_Renta = df_INE_Estructura_Datos_Renta[~df_INE_Estructura_Datos_Renta["Nombre"].str.contains("sección|distrito")]
    df_INE_Estructura_Datos_Seccion_Renta = df_INE_Estructura_Datos_Renta[df_INE_Estructura_Datos_Renta["Nombre"].str.contains("sección")]
    df_INE_Estructura_Datos_Distrito_Renta = df_INE_Estructura_Datos_Renta[df_INE_Estructura_Datos_Renta["Nombre"].str.contains("distrito")]
    
    df_INE_Estructura_Datos_Total_Renta["Tipo"] = "Total"
    df_INE_Estructura_Datos_Total_Renta = df_INE_Estructura_Datos_Total_Renta[["Id","COD","Nombre","Tipo","Tipo Renta"]]
    #display(df_INE_Estructura_Datos_Total_Renta)
    
    new2 = df_INE_Estructura_Datos_Seccion_Renta["Nombre"].str.split('sección')
    df_INE_Estructura_Datos_Seccion_Renta["Tipo"] = new2.map(lambda x: "sección " + x[1].strip())
    df_INE_Estructura_Datos_Seccion_Renta["Nombre"] = new2.map(lambda x: x[0].strip())
    df_INE_Estructura_Datos_Seccion_Renta = df_INE_Estructura_Datos_Seccion_Renta[["Id","COD","Nombre","Tipo","Tipo Renta"]]
    #display(df_INE_Estructura_Datos_Seccion_Renta)
    
    new3 = df_INE_Estructura_Datos_Distrito_Renta["Nombre"].str.split('distrito')
    df_INE_Estructura_Datos_Distrito_Renta["Tipo"] = new3.map(lambda x: "distrito " + x[1].strip())
    df_INE_Estructura_Datos_Distrito_Renta["Nombre"] = new3.map(lambda x: x[0].strip())
    df_INE_Estructura_Datos_Distrito_Renta = df_INE_Estructura_Datos_Distrito_Renta[["Id","COD","Nombre","Tipo","Tipo Renta"]]
    #display(df_INE_Estructura_Datos_Distrito_Renta)
    
    df_INE_Estructura_Datos_Renta_New = pd.concat([df_INE_Estructura_Datos_Total_Renta, df_INE_Estructura_Datos_Seccion_Renta, df_INE_Estructura_Datos_Distrito_Renta], axis=0)
    df_INE_Estructura_Datos_Renta_New = df_INE_Estructura_Datos_Renta_New.sort_values(by=['Nombre', "Tipo"])
    #display(df_INE_Estructura_Datos_Renta_New)
    #df_INE_Estructura_Datos_Renta_New.to_excel("df_INE_Estructura_Datos_Renta_New.xlsx")
    
    for index, poblacion in df_INE_Estructura_Datos_Renta_New.iterrows():
        codigo = poblacion["COD"]
        url_datos = url_plantilla.format(codigo=codigo, num_datos=num_datos)
        df_renta_ = pd.DataFrame.from_dict(requests.get(url_datos).json()["Data"])
        
        if len(list(df_renta_.columns)) > 0:
            df_renta_ = df_renta_[["Anyo","Fecha","Valor"]].rename(columns={"Anyo": "Año", "Fecha": "Fecha", "Valor": poblacion["Tipo Renta"].strip()})
            df_renta_["Fecha"] = pd.to_datetime(df_renta_["Fecha"], unit='ms')
            df_renta_["Fecha"] = df_renta_["Fecha"].map(lambda x: x + hours_added)
            df_renta_['Mes'] = df_renta_['Fecha'].dt.month
            df_renta_ = df_renta_[["Año", "Mes", poblacion["Tipo Renta"].strip()]]
            df_renta_ = df_renta_.set_index(["Año", "Mes"])
            #display(df_renta_)
            
            df_renta_.columns = pd.MultiIndex.from_product([ [provincia], [poblacion["Nombre"].strip()], [poblacion["Tipo"].strip()], df_renta_.columns], names=["Provincia", "Población","Tipo", "Tipo Renta"])
            
            df_renta = pd.concat([df_renta, df_renta_], axis=1)

C:\Users\abediaga\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\abediaga\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\abediaga\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [64]:
display(df_renta)

Provincia                       Bizkaia                             \
Población                       Abadiño                              
Tipo                              Total                              
Tipo Renta Renta neta media por persona Renta neta media por hogar   
Año  Mes                                                             
2015 1                          13919.0                    35774.0   
2016 1                          14515.0                    36912.0   
2017 1                          14724.0                    37078.0   
2018 1                              NaN                        NaN   

Provincia                                                           \
Población                                                            
Tipo                        distrito 01                              
Tipo Renta Renta neta media por persona Renta neta media por hogar   
Año  Mes                                                             
2015 1                          13919.0                    35774.0   
2016 1                          14515.0                    36912.0   
2017 1                          14724.0                    37078.0   
2018 1                              NaN                        NaN   

Provincia                                                           \
Población                                                            
Tipo                      sección 01001                              
Tipo Renta Renta neta media por persona Renta neta media por hogar   
Año  Mes                                                             
2015 1                          15658.0                    40113.0   
2016 1                          16437.0                    42664.0   
2017 1                          16294.0                    41896.0   
2018 1                              NaN                        NaN   

Provincia                                                           \
Población                                                            
Tipo                      sección 01002                              
Tipo Renta Renta neta media por persona Renta neta media por hogar   
Año  Mes                                                             
2015 1                          12855.0                    32698.0   
2016 1                          13468.0                    33452.0   
2017 1                          13756.0                    33455.0   
2018 1                              NaN                        NaN   

Provincia                                                           ...  \
Población                                                           ...   
Tipo                      sección 01003                             ...   
Tipo Renta Renta neta media por persona Renta neta media por hogar  ...   
Año  Mes                                                            ...   
2015 1                          11455.0                    28190.0  ...   
2016 1                          11908.0                    29114.0  ...   
2017 1                          12159.0                    29340.0  ...   
2018 1                              NaN                        NaN  ...   

Provincia                         Araba                             \
Población                 Yécora/Iekora                              
Tipo                              Total                              
Tipo Renta Renta neta media por persona Renta neta media por hogar   
Año  Mes                                                             
2015 1                          11582.0                    28620.0   
2016 1                          12297.0                    30205.0   
2017 1                          12808.0                    31709.0   
2018 1                          13722.0                    34171.0   

Provincia                                                           \
Población                     Zalduondo                              
Tipo                         

In [65]:
df_renta.to_excel("renta_Euskadi.xlsx")